In [131]:
import pandas as pd
import re
from collections import Counter

pattern1 = re.compile(r'[+/]')
pattern2 = re.compile(r'\(\w+\)')

def split_and_clean_course_code(l):
    if type(l) != str:
        return []
    tmp = pattern1.sub(r',', l)
    tmp = pattern2.sub(r'', tmp)
    tmp = tmp.strip().strip(',')
    tmp = [c.strip() for c in tmp.split(',')]
    return tmp

In [132]:
Projections = {
                'Old': ['StudentsAudit_Fatna.xlsm','STUD_ID','Manual Audit'],
                'New': ['StudentsAudit.xlsx','ID','Audit_Courses_to_add'],
                # 'New': ['StudentsAudit_old.xlsx','ID','Audit_Courses_to_add']
}

In [133]:
old = pd.read_excel(Projections['Old'][0])[Projections['Old'][1:]]
new = pd.read_excel(Projections['New'][0])[Projections['New'][1:]]
old.columns = ['ID','old']
new.columns = ['ID','new']
old['oID'] = old.ID
new['nID'] = new.ID
df = pd.merge(old, new, on='ID', how='outer')

count = len(df)
mask1 = df.oID.isnull() & df.nID.notnull()
mask2 = df.oID.notnull() & df.nID.isnull()
count1 = len(df[mask1])
count2 = len(df[mask2])
if count1:
    print(f'Found {count1} in new but missing from old ({count1/count*100:.2f}%). Excluding the records.')
    # print(df[mask1])
    df.drop(df[mask1].index, inplace=True)
if count2:
    print(f'\nFound {count2} in old but missing from new ({count2/count*100:.2f}%). Excluding the records.')
    # print(df[mask2])
    df.drop(df[mask2].index, inplace=True)
df.drop(['oID','nID'],inplace=True,axis=1)

print(f'\n{len(df)} records left after cleaning.\n')

Found 642 in new but missing from old (39.65%). Excluding the records.

977 records left after cleaning.



In [134]:
df.old = df.old.apply(split_and_clean_course_code)
df.new = df.new.apply(split_and_clean_course_code)
df['Delta_old'] = pd.Series(dtype='object')
df['Delta_new'] = pd.Series(dtype='object')
df['Match'] = pd.Series(dtype='float')

for i, r in df.iterrows():
    o = Counter(r.old)
    n = Counter(r.new)
    Delta_o_n = list((o-n).elements())
    Delta_n_o = list((n-o).elements())
    df.at[i,'Delta_old'] = Delta_o_n
    df.at[i,'Delta_new'] = Delta_n_o
    if len(r.old)+len(r.new) == 0:
        df.at[i,'Match'] = 1
    else:
        df.at[i,'Match'] = 1 - (len(Delta_o_n)+len(Delta_n_o)) / (len(r.old)+len(r.new))

print(f'\ncompared {len(df)} students, with similarity ratio: {df.Match.sum()/len(df)*100:.2f}%')
print(f'\n{len(df[ (df.Delta_old.map(len)>0) | (df.Delta_new.map(len)>0) ])} differences:')
# display(df[ (df.Delta_old.map(len)>0) | (df.Delta_new.map(len)>0) ])


compared 977 students, with similarity ratio: 94.45%

159 differences:


compared 977 students, with similarity ratio: 94.45%, 159 differences -> max 16 credit hours

compared 977 students, with similarity ratio: 94.69%, 157 differences -> controlled duplicates

compared 977 students, with similarity ratio: 94.48%, 170 differences -> key courses by concentration

compared 977 students, with similarity ratio: 94.18%, 177 differences -> Baseline

In [135]:
# display(df[ (df.Delta_old.map(len)>0) | (df.Delta_new.map(len)>0) ][0:50])
# display( df[df['old'].astype(str).str.contains(')',regex=False)] )

In [136]:
cat = ['Status', 'Standing', 'Campus', 'Audit_UsedCatalog', 'College', 'Degree', 'Major', 'Concentration', 'Program']

data = pd.read_excel(Projections['New'][0])
df2 = pd.merge(data, df, on='ID', how='inner')
for c in cat:
    display(df2.groupby(c)['Match'].agg(['count','mean']))

,count,mean
Status,,
Academic Dismissal (DS),4,0.840909
Active (AS),865,0.944874
Attendance Suspension (AT),3,0.933333
Maternity Suspension (MA),2,1.000000
Military Service (MS),12,0.966667
Military Svc Completion (MC),58,1.000000
Military Svc Exempted (ME),13,1.000000
Registration Suspension (RS),9,0.835017
Reinstated (RE),1,0.000000


,count,mean
Standing,,
Academic Dismissal (AD),4,0.840909
Academic Probation 1 (P1),5,0.661818
Academic Probation 1 (PO),4,0.375000
Academic Probation 2 (P2),10,0.723182
Academic Probation 2 (PT),4,0.791667
Academic Warning (AW),62,0.948663
Academic Warning 1 (W1),16,0.793087
Academic Warning 2 (W2),1,0.400000
Good Standing (00),6,0.833333


,count,mean
Campus,,
AUH - Men's Campus,164,0.967568
AUH - Women's Campus,181,0.979549
AUH Mixed Gender,632,0.928402


,count,mean
Audit_UsedCatalog,,
2019,58,0.971787
2020,139,0.976091
2021,337,0.967530
2024,303,0.993399
2024_CIS,139,0.745553


,count,mean
College,,
Technological Innovation (TI),977,0.944452


,count,mean
Degree,,
Bachelor of Science (BS),977,0.944452


,count,mean
Major,,
Info Systems & Tech Management (BIST),278,0.931619
Information Technology (BINT),699,0.949556


,count,mean
Concentration,,
Business Intelligence (_BI),201,0.926861
Enterprise Systems (_ES),6,0.966667
Mgmt. of Information Systems (_MS),71,0.942125
Security and Network Tech (_SN),637,0.947123
Web and Mobile App Development (_WM),62,0.974560


,count,mean
Program,,
TI_BSINT,699,0.949556
TI_BSIST,278,0.931619


In [137]:
df2.to_excel('Compare_Projections.xlsx')


In [138]:
df2[(df2['Audit_UsedCatalog']=='2024') & (df2['Match']<0.9)]

,Unnamed: 0,ID,Name,Email,Status,Standing,Campus,Admitted Term,Catalog,College,...,Audit_CH_registered,Audit_Exception_Mgs,Audit_Applied_Rules,Audit_Courses_to_add_extend,Audit_Courses_to_add_CH_extend,old,new,Delta_old,Delta_new,Match
219,361,202301285,Faris Mubarak Salem Omar Almenhali,202301285@zu.ac.ae,Active (AS),Good Standing (G1),AUH Mixed Gender,202321,202421,Technological Innovation (TI),...,0.0,[],CIT490 can only be one of the last two courses...,"CIT365, ARA345, INS377, CIT480, ZU_Elective",15.0,"[MTH215, ENG240, SEC235, ZU_Elective]","[MTH215, ENG240, SEC235, ZU_Elective, NET256, ...",[],"[NET256, NET257]",0.8
742,1253,202420015,Sara Khalid Nasser Alshafi Alhajeri,202420015@zu.ac.ae,Active (AS),NaN,AUH Mixed Gender,202422,202422,Technological Innovation (TI),...,0.0,[],CIT490 can only be one of the last two courses...,"INS260, MTH215, ENG240, MGT209, MKT210",15.0,"[FYE120, FYE150, FYE105, FYE195, FYE146]","[FYE120, FYE150, FYE105, FYE195, FYE145]",[FYE146],[FYE145],0.8
844,1431,202411995,Wafa Muhammad Farooq Alhashmi,202411995@zu.ac.ae,Active (AS),Good Standing (G1),AUH Mixed Gender,202421,202421,Technological Innovation (TI),...,0.0,[],CIT490 can only be one of the last two courses...,"ZU_Elective, ARA345, CIT480, ZU_Elective",12.0,"[FYE146, MTH215, CIT210, ENG240, SEC235]","[FYE145, CIT210, MTH215, ENG240, SEC235]",[FYE146],[FYE145],0.8
875,1474,202410972,Gaith Khalifa Ateeq Mohamed Almazrouei,202410972@zu.ac.ae,Military Service (MS),NaN,AUH Mixed Gender,202422,202422,Technological Innovation (TI),...,0.0,[],CIT490 can only be one of the last two courses...,"FYE105, FYE125, FYE155, FYE195, FYE145",15.0,"[FYE100, FYE110, FYE140, FYE150, FYE105]","[FYE100, FYE110, FYE120, FYE140, FYE150]",[FYE105],[FYE120],0.8
877,1476,202414935,Mohamed Awadh Brek Salem Aljaberi,202414935@zu.ac.ae,Military Service (MS),NaN,AUH Mixed Gender,202422,202422,Technological Innovation (TI),...,0.0,[],CIT490 can only be one of the last two courses...,"FYE105, FYE125, FYE155, FYE195, FYE145",15.0,"[FYE100, FYE110, FYE105, FYE140, FYE150]","[FYE100, FYE110, FYE120, FYE140, FYE150]",[FYE105],[FYE120],0.8
972,1611,202304452,Rashed Ashraf Khamis Jabr Alsuwaidi,202304452@zu.ac.ae,Withdrawn (WD),Good Standing (00),AUH Mixed Gender,202321,202422,Technological Innovation (TI),...,0.0,[],CIT490 can only be one of the last two courses...,"ENG240, SEC235, ZU_Elective, ARA345, CIT480",15.0,[withdrawn],"[FYE105, FYE195, FYE145, CIT210, MTH215]",[withdrawn],"[FYE105, FYE195, FYE145, CIT210, MTH215]",0.0
